In [29]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import pickle

In [58]:
@njit
def powerlaw_binned_spectrum(energy_bins, spectrum):
    assert np.amin(energy_bins) > 0, "All energy bin values must be greater 0"
    assert np.amin(spectrum) > 0, "All spectrum values must be greater 0"
    
    B = np.log(spectrum[1:] / spectrum[:-1]) / np.log(energy_bins[1:] / energy_bins[:-1])
    A = spectrum[:-1] / (energy_bins[:-1] ** B)
    
    C = B+1.
    # prevent rounding errors
    C[np.abs(C) < 1e-7] = 0
    
    regular = np.nonzero(C)[0]
    non_regular = []
    for i in range(len(energy_bins) - 1):
        if not i in regular:
            non_regular.append(i)
    non_regular = np.array(non_regular)
    print(non_regular)
    
    binned_spectrum = np.zeros(len(energy_bins) - 1)
    binned_spectrum[regular] = A[regular] / (C[regular]) * (energy_bins[regular+1]**(C[regular]) - energy_bins[regular]**(C[regular]))
    binned_spectrum[non_regular] = A[non_regular] * (np.log(energy_bins[non_regular+1]) - np.log(energy_bins[non_regular]))
    
    return binned_spectrum
    

In [61]:
a = np.linspace(1,11,11)
b = np.linspace(11,1,11)
b[1]=11*2**(-1-1.0e-7)


In [62]:
powerlaw_binned_spectrum(a,b)

[]


array([7.62461872, 7.22479233, 8.46626555, 7.47031054, 6.47198726,
       5.47198726, 4.47031054, 3.46626555, 2.45755523, 1.43482524])

In [63]:
(a[1:]-a[:-1])*(b[:-1]+b[1:])/2

array([8.24999981, 7.24999981, 8.5       , 7.5       , 6.5       ,
       5.5       , 4.5       , 3.5       , 2.5       , 1.5       ])

In [7]:
(3**-2.22044605e-16 - 2 ** -2.22044605e-16)

-1.1102230246251565e-16

In [8]:
11 * ( np.log(3) - np.log(2))

4.46011618918981

In [14]:
g = 6**-2.22044605e-16

In [15]:
h = 5 ** -2.22044605e-16

In [16]:
g- h

-1.1102230246251565e-16